In [7]:
import pandas as pd
import numpy as np
from wordcloud import STOPWORDS

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import to_categorical
from keras import regularizers
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers

%matplotlib inline

In [5]:
senate = pd.read_csv('./data/senate_processed.csv')

In [16]:
senate.dropna(inplace=True)

In [9]:
senate['ratio_quartile'] = pd.qcut(senate['the_ratio'], 4, labels=False)

In [10]:
senate['ratio_quartile'].value_counts()

1    59428
0    54906
3    54855
2    50350
Name: ratio_quartile, dtype: int64

In [11]:
# establishing our baseline accuracy
baseline_accuracy = senate['ratio_quartile'].value_counts()[0]/len(senate['ratio_quartile'])
baseline_accuracy

0.2500967937359649

In [12]:
stopwords = set(STOPWORDS)
stopwords.add("http")
stopwords.add("co")
stopwords.add("amp")
stopwords.add("u")
stopwords.add("w")
stopwords.add("bit")
stopwords.add("ly")

In [36]:
# Setting our X,y and encoding our target variable
y = senate['ratio_quartile']
X = senate['text_processed']

cv = CountVectorizer(stop_words=stopwords, strip_accents='unicode')
cv.fit(X)
X = cv.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=42)

le = LabelEncoder()
le.fit(y)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [29]:
y_train = y_train.reshape(-1,1)


In [31]:
y_train.shape

(153675, 1)

In [33]:
model = Sequential()

n_input = X_train.shape[1]
n_hidden = n_input
n_output = y_train.shape[1]

model.add(Dense(n_hidden,
                  input_dim= n_input,
                  activation='relu'))
model.add(Dropout(.2))

model.add(Dense(n_output,
                  activation='softmax'))

model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [34]:

hist = model.fit(X_train,y_train,
                 validation_data=(X_test,y_test),
                 epochs= 100,
                 verbose=1)

ValueError: You are passing a target array of shape (153675, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.